In [2]:
import os
print(os.getcwd())
print(os.listdir())

/home/shasm/what-do-i-play-next/data
['raw', 'API_call_nb.ipynb', 'API_call.py', '__init__.py', 'README.md', 'processed', 'scraper.py', '__pycache__']


In [ ]:
import inspect
import imslp 
import sys
sys.path.append("..")  # or project root

from scraper import scrape_imslp_metadata




{'Key': 'B major', 'Year/Date of Composition': '2021', 'Average Duration': '1 minute', 'Composer Time Period': 'Modern', 'Piece Style': 'Modern', 'Instrumentation': 'piano 4-hands'}


In [ ]:
from imslp.interfaces import internal

works = internal.list_works(count=8000)
for i, r in enumerate(works, 1):
    title = r.get('intvals', {}).get('worktitle')
    composer = r.get('intvals', {}).get('composer')
    permlink = r.get('permlink')
    print(f"{i}. {title} — composer: {composer} — permlink: {permlink}")
try:
    total = len(works)
except TypeError:
    works = list(works)
    total = len(works)
print("total:", total)

1. "Amicizia" — composer: Stankovych, Tatiana — permlink: https://imslp.org/wiki/"Amicizia"_(Stankovych,_Tatiana)
2. "Los ojos tiernos" — composer: Esnaola, Juan Pedro — permlink: https://imslp.org/wiki/"Los_ojos_tiernos"_(Esnaola,_Juan_Pedro)
3. "Mrs. Ramsay rose. Lily rose." — composer: Eisenbrey, Keith — permlink: https://imslp.org/wiki/"Mrs._Ramsay_rose._Lily_rose."_(Eisenbrey,_Keith)
4. "My Old Dutch" Waltz — composer: West, Alfred H. — permlink: https://imslp.org/wiki/"My_Old_Dutch"_Waltz_(West,_Alfred_H.)
5. "Sombras de Jerez"- Artiom Galuza — composer: Galuza, Artiom — permlink: https://imslp.org/wiki/"Sombras_de_Jerez"-_Artiom_Galuza_(Galuza,_Artiom)
6. "Tagh of Awakening" — composer: Hakobjanyan, Anna — permlink: https://imslp.org/wiki/"Tagh_of_Awakening"_(Hakobjanyan,_Anna)
7. "The Girl from the Bus Station" — composer: Gonzaga, Thales — permlink: https://imslp.org/wiki/"The_Girl_from_the_Bus_Station"_(Gonzaga,_Thales)
8. "Trip-O-Let" Dance — composer: Hamilton, Jesse J. — p

In [6]:
from imslp.interfaces import internal
from imslp.client import ImslpClient
from imslp.interfaces.scraping import fetch_images_metadata

works = internal.list_works(count=10)
if not works:
    raise SystemExit("no works returned")

first = next((r for r in works if r.get('permlink')), works[0])
permlink = first['permlink']
print("Using permlink:", permlink)

client = ImslpClient()
page = client._site.pages[permlink]
print("page title:", getattr(page, "base_title", None))

imgs = fetch_images_metadata(page)
print("images found:", len(imgs))
for m in imgs[:5]:
    print({k: m.get(k) for k in ("name", "url", "download_count") if k in m})

Using permlink: https://imslp.org/wiki/"Amicizia"_(Stankovych,_Tatiana)
page title: Https:
images found: 0


In [7]:
import csv
def file_scrape(work):
    row = {}

    row["Title"] = work.get("intvals", {}).get("worktitle")
    row["Composer"] = work.get("intvals", {}).get("composer")
    row["Permlink"] = work.get("permlink")

    try:
        additional_info = scrape_imslp_metadata(row["Permlink"])

        row["Instrumentation"] = additional_info.get("Instrumentation")
        row["Key"] = additional_info.get("Key")
        row["Piece Style"] = additional_info.get("Piece Style")
        row["First Performance"] = additional_info.get("First Performance")
        row["Year/Date of Composition"] = additional_info.get("Year/Date of Composition")
        row["Composer Time Period"] = additional_info.get("Composer Time Period")
        row["Average Duration"] = additional_info.get("Average Duration")

    except Exception as e:
        print(f"Scrape failed for {row['Permlink']}: {e}")

        row["Instrumentation"] = None
        row["Key"] = None
        row["Piece Style"] = None
        row["First Performance"] = None
        row["Year/Date of Composition"] = None
        row["Composer Time Period"] = None
        row["Average Duration"] = None
    return row



In [ ]:
#test
i = 0
while (i < 9):
    work = works[i]
    row = file_scrape(work)
    print(row)
    i += 1


{'Title': '"Amicizia"', 'Composer': 'Stankovych, Tatiana', 'Permlink': 'https://imslp.org/wiki/"Amicizia"_(Stankovych,_Tatiana)', 'Instrumentation': 'piano 4-hands', 'Key': 'B major', 'Piece Style': 'Modern', 'First Performance': None, 'Year/Date of Composition': '2021', 'Composer Time Period': 'Modern', 'Average Duration': '1 minute'}
{'Title': '"Los ojos tiernos"', 'Composer': 'Esnaola, Juan Pedro', 'Permlink': 'https://imslp.org/wiki/"Los_ojos_tiernos"_(Esnaola,_Juan_Pedro)', 'Instrumentation': 'piano', 'Key': 'F major', 'Piece Style': 'Romantic', 'First Performance': None, 'Year/Date of Composition': '1837', 'Composer Time Period': 'Romantic', 'Average Duration': '2 minutes'}
{'Title': '"Mrs. Ramsay rose. Lily rose."', 'Composer': 'Eisenbrey, Keith', 'Permlink': 'https://imslp.org/wiki/"Mrs._Ramsay_rose._Lily_rose."_(Eisenbrey,_Keith)', 'Instrumentation': 'piano', 'Key': None, 'Piece Style': 'Modern', 'First Performance': None, 'Year/Date of Composition': '2001', 'Composer Time Per

In [ ]:
import csv
import time

FIELDNAMES = [
    "Title",
    "Composer",
    "Permlink",
    "Instrumentation",
    "Key",
    "Piece Style",
    "First Performance",
    "Year/Date of Composition",
    "Composer Time Period",
    "Average Duration",
]

def populate_csv(works, output_path, sleep_seconds=0.5):
    """
    Populate a CSV file from a list of IMSLP works.

    works: iterable of work dicts (from internal.list_works)
    output_path: path to CSV file
    sleep_seconds: delay between scrapes (rate limiting)
    """

    with open(output_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=FIELDNAMES)
        writer.writeheader()

        for idx, work in enumerate(works, 1):
            try:
                row = file_scrape(work)
                writer.writerow(row)

                print(f"[{idx}] Wrote: {row['Title']}")

            except Exception as e:
                print(f"[{idx}] Failed row: {e}")

            # 🚦 RATE LIMITING (important)
            time.sleep(sleep_seconds)


In [31]:
internal.load_cache(from_file=True)

# 2. Access the cached works
cached_works = internal.list_works(cache=True)  # use cache=True to read from memory
print(len(cached_works))


6000


In [18]:
populate_csv(works, "raw.csv")

[1] Wrote: "Amicizia"
[2] Wrote: "Los ojos tiernos"
[3] Wrote: "Mrs. Ramsay rose. Lily rose."
[4] Wrote: "My Old Dutch" Waltz
[5] Wrote: "Sombras de Jerez"- Artiom Galuza
[6] Wrote: "Tagh of Awakening"
[7] Wrote: "The Girl from the Bus Station"
[8] Wrote: "Trip-O-Let" Dance
[9] Wrote: "Und ich sah ...", Op.59
[10] Wrote: "Urlicht"
[11] Wrote: "of being a self in a song"
[12] Wrote: "s'kommt ein Vogel geflogen"
[13] Wrote: "un solo quarto d'ora"
[14] Wrote: $naFU
[15] Wrote: '12' Short Pieces
[16] Wrote: '4-4-8' Claves Quartet
[17] Wrote: 'A Grotta 'e Pozzulo
[18] Wrote: 'A Margarita
[19] Wrote: 'A Napulitana
[20] Wrote: 'A bella d''e belle
[21] Wrote: 'A buscia
[22] Wrote: 'A cafettera
[23] Wrote: 'A cafettera
[24] Wrote: 'A cafona
[25] Wrote: 'A cagnacavalle
[26] Wrote: 'A calamita
[27] Wrote: 'A cammisa affatata
[28] Wrote: 'A cammurrestella
[29] Wrote: 'A canzone d''e cucchiere
[30] Wrote: 'A capera
[31] Wrote: 'A cartulina
[32] Wrote: 'A casa d''e fate
[33] Wrote: 'A cchiù bella fi

KeyboardInterrupt: 